## Created by <a href="https://github.com/yunsuxiaozi/">yunsuxiaozi</a> 2024/8/17

#### 比赛网址:<a href="https://challenge.xfyun.cn/topic/info?type=intrusion-detection&option=phb&ch=vWxQGFU">网络安全入侵检测挑战赛</a>,目前排名为2/46。距离比赛还有3天结束,整场比赛打下来感觉没什么意思,也学不到什么东西,后面不打算再继续卷这场比赛了。开源供想要上分的小伙伴学习。

In [1]:
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
from tqdm import tqdm#加载进度条的库
import gc
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

import random#提供了一些用于生成随机数的函数
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=2024)

In [2]:
path=''#你的文件夹路径
train=pd.read_csv(path+"train_data.csv")
test=pd.read_csv(path+"test_data.csv")
print(f"len(train):{len(train)},len(test):{len(test)}")
#feat_8和feat_9相关性为1,故去掉feat_9,  
#1 7    11 23    13 16    39 40    14 17相似度也高达0.99
#feat_1和feat_37有0.97的相关性,feat_34和feat_39也有0.9885的相关性
#id不具有实际意义,feat_31和每列都不相关,feat_18和label相关性只有万分之一
train.drop(['id','feat_9','feat_7','feat_16','feat_23','feat_40','feat_31','feat_37','feat_39','feat_18','feat_17'],axis=1,inplace=True)
test.drop(['id','feat_9','feat_7','feat_16','feat_23','feat_40','feat_31','feat_37','feat_39','feat_18','feat_17'],axis=1,inplace=True)
train.head()

len(train):800000,len(test):200000


,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_8,feat_10,feat_11,...,feat_30,feat_32,feat_33,feat_34,feat_35,feat_36,feat_38,feat_41,feat_42,label
0,6,0,656,0,0,140,0,35840,5612,0,...,7.00,0,0,0,0,0,0,64,0,0
1,17,0,0,1120000,70,0,1,0,57621,0,...,0.00,0,29,0,2,2040000,55113,128,1,1
2,17,0,921,224000,0,0,0,0,54105,0,...,188.00,0,0,0,2,0,0,64,0,2
3,6,0,0,352000,40,0,2,0,49546,20,...,0.00,0,0,0,2,320000,0,0,0,1
4,17,0,0,456000,57,0,3,0,54162,0,...,5.68,0,223,0,1,1160000,14256,0,28,1


In [3]:
feat_one_hot=[['feat_0',[6,17,1,2,58]],#feat_0除了这5个以外,剩下的都是出现10次以下的数值
 #feat_1和feat_2像是count的统计,其中feat_2太多了
 ['feat_1',[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 20, 22, 24, 28, 30, 32, 36, 40, 42, 44, 48, 54, 60, 66, 352, 704, 720]],#出现100次以上的
 ['feat_2',[0, 16, 15, 1000, 1125, 1250, 875, 1375, 953, 1078, 1047, 1172, 1203, 1297, 828, 31, 922, 750, 1328, 1031, 1219, 1422, 969, 1156, 844, 625, 1281, 1266, 672, 703, 1500, 1344, 906, 1094, 500, 797, 1141, 1359, 1453, 47, 719, 859, 1016, 891, 453, 781, 1406, 422, 656, 1234, 1188, 1109, 578, 375, 766, 984, 547, 1391, 938, 469, 1187, 1469, 609, 531, 594, 937, 813, 641, 406, 1625, 328, 734, 1062, 1063, 1578, 297, 1312, 3, 812, 1547, 688, 1484, 484, 1750, 1313, 1265, 1438, 281, 1531, 1235, 1437, 391, 985, 344, 890, 1594, 687, 562, 516, 1703]],#挑选前100个特征
 ['feat_3',[1120000, 224000, 352000, 0, 384000, 320000, 544000, 520000, 16000, 8000, 1056000, 560000, 568000, 24000, 536000, 504000, 480000, 552000, 512000, 896000, 11840000, 1168000, 576000, 11680000, 472000, 584000, 528000, 11808000, 40000, 32000, 488000, 632000, 720000, 416000, 2984000, 3984000, 4680000, 496000, 80000, 1040000, 464000, 600000, 592000, 104000, 88000, 1024000, 96000, 944000, 960000, 1088000]],#挑选前50个特征
 #feat_3%8000=0,除了一个数据
 ['feat_4',[0, 40, 44, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 88, 89, 90, 92, 93, 96, 97, 102, 104, 105, 106, 111, 112]],#出现100次以上的
 ['feat_5',[0, 41, 74, 82, 83, 100, 118, 123, 125, 126, 128, 132, 136, 140, 143, 164, 166, 174, 177, 180, 187, 190, 195, 201, 205, 207, 208, 219, 239, 245, 246, 249, 251, 257, 261, 280, 285, 287, 289, 293, 299, 302, 303, 304, 305, 306, 307, 309, 310, 328, 329, 332, 348, 351, 354, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 390, 392, 393, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 416, 417, 418, 419, 420, 421, 422, 423, 424, 428, 429, 430, 431, 432, 433, 436, 451, 456, 462, 490, 492, 505, 506, 533, 539, 547, 555, 563, 574, 578, 605, 615, 621, 623, 626, 642, 644, 656, 658, 663, 674, 688, 690, 697, 701, 706, 711, 712, 713, 717, 722, 738, 779, 782, 784, 786, 795, 800, 820, 826, 842, 850, 875, 891, 920, 925, 945, 946, 952, 962, 989, 1140, 1340, 1402, 1434, 1435, 1472, 1541, 1660, 1665, 1676, 1679, 1680, 1684, 1692, 1712, 1744, 1752, 1756, 1764, 1768, 1812, 1820, 1836, 1860, 1868, 1912, 1968, 1980, 1982, 1992, 2024, 2028, 2064, 2068, 2076, 2088, 2132, 2140, 2234, 2235, 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2632, 2820, 2852, 2961, 3144, 3240, 3537, 4025, 4252, 4812, 4948, 5244, 5744, 8440, 8944, 18117, 18118, 18119, 18123, 18133, 18138, 18144, 18149, 18159, 18160, 18162, 18194, 18208, 18223, 18282]],#大于100
 ['feat_6',[5, 0, 4, 1, 6, 36, 39, 3, 9, 41, 2, 8, 22, 12, 35, 70, 60, 25, 80, 45, 58, 16, 13, 14, 47, 76, 38, 42, 30, 67, 61, 28, 32, 105, 75, 71, 98, 403, 127, 170, 113, 174, 62, 133, 321, 276, 153, 104, 375, 172]],#挑选前50个特征
 ['feat_8',[0, 35840, 49920, 6912, 6144, 8960, 11008, 16896, 62976, 13056, 52480, 33792, 64000, 63744, 48640, 44544, 10496, 39936, 31232, 46080, 32000, 18944, 21248, 13568, 64256, 1280, 32256, 20992, 41984, 62720, 36864, 36608, 48384, 8448, 50944, 15360, 31488, 18432, 26112, 2048, 38656, 29696, 12032, 39424, 7936, 28416, 9216, 59392, 60416, 37120, 53248, 49152, 3328, 256, 48128, 25600, 29184, 2816, 28928, 49408, 48896, 49664, 43008, 1024, 26368, 31744, 45056, 28672, 27392, 27904, 27136, 53504, 45568, 13824, 52224, 21504, 26624, 47872, 23808, 4096, 37376, 8192, 51200, 5376, 39680, 11776, 45312, 38912, 47360, 47616, 11264, 12800, 61440, 12288, 41728, 61952, 37632, 55296, 18688, 63488]],#挑选前100个特征
 ['feat_10',[443, 80, 57044, 42640, 52071, 56304, 49731, 40081, 59460, 65233, 57184, 41607, 47422, 47095, 54114, 50428, 35063, 40850, 0, 54193, 53, 52970, 43411, 50099, 50519, 51598, 51103, 50227, 44486, 6000, 53238, 49572, 57121, 50063, 36073, 36072, 57673, 54875, 51694, 52695, 55851, 33002, 58578, 52089, 64935, 61417, 52160, 54572, 57720, 49350]],#挑选前50个特征            
 ['feat_11',[0, 2, 4, 6, 16, 17, 18, 19, 20, 22, 24, 25, 26, 27, 30, 31, 82, 86, 90, 91, 94, 95, 218]],#feat_11>100次
 ['feat_12',[28, 140, 44, 1500, 48, 60, 1213, 40, 975, 52, 81, 1189, 89, 1352, 1470, 987, 106, 100, 86, 68, 65, 1219, 1004, 71, 167, 67, 1406, 180, 171, 528, 95, 84, 130, 88, 117, 134, 80, 107, 91, 1195, 127, 156, 746, 449, 87, 129, 99, 94, 160, 97]],#挑选前50个特征
 ['feat_13',[6, 2, 5, 0, 13, 21, 10, 64, 86, 3, 30, 33, 47, 111, 138, 44, 77, 31, 74, 211, 55, 15, 69, 39, 20, 136, 61, 9, 101, 66, 114, 108, 198, 185, 273, 244, 104, 150, 4, 129, 160, 12, 72, 52, 14, 501, 573, 650, 301, 90]],#挑选前50个特征          
 ['feat_14',[0, 2, 4, 6, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 41, 194, 211, 214, 219, 222, 223]],#feat_14>100次
 ['feat_15',[0, 1, 2, 3, 4, 5, 6, 7, 8, 18, 32, 64]],#feat_15>100个数
 ['feat_19',[0.0, 125.0, 221.0, 229.0, 230.0, 331.0, 530.0]],#大于100
 ['feat_20',[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 16, 17, 18, 20, 29, 37]],#大于100
 ['feat_21',[0.0, 40.0, 1873.0, 44.0, 1147.0, 112.0, 132.0, 81.0, 1721.0, 100.0, 68.0, 65.0, 1136.0, 86.0, 178.0, 71.0, 67.0, 167.0, 120.0, 60.0, 700.0, 2143.0, 344.0, 764.0, 342.0, 95.0, 80.0, 164.0, 88.0, 84.0, 52.0, 323.0, 89.0, 130.0, 180.0, 162.0, 172.0, 92.0, 10168.0, 240.0, 334.0, 6015.0, 128.0, 107.0, 156.0, 1913.0, 91.0, 1869.0, 304.0, 160.0]],#取前50个value
 ['feat_22',[2, 1, 3, 5, 4, 8, 6, 10, 9, 7, 14, 12, 11, 13, 17, 16, 15, 18, 19, 20, 22, 21, 27, 23, 26, 24, 52, 28, 100, 25, 33, 44, 48, 122, 45, 37, 40, 43, 60, 42, 34, 38, 29, 46, 32, 54, 36, 200, 56, 72]],#取前50个value  
 ['feat_24',[28, 140, 44, 1500, 48, 60, 1213, 40, 975, 52, 81, 1189, 89, 1352, 1470, 987, 106, 100, 86, 68, 65, 1219, 1004, 71, 167, 67, 1406, 180, 171, 528, 95, 84, 130, 88, 117, 134, 80, 107, 91, 1195, 127, 156, 746, 449, 87, 129, 99, 94, 160, 97]],#取前50个value  
              
 ['feat_25',[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 38, 39, 41, 43, 46, 47, 62, 63, 78, 79, 93, 94, 109, 110, 125, 140, 141, 156, 157, 172, 187, 188, 203, 218, 219, 234, 235, 250, 265, 266, 281, 282, 297, 312, 313, 328, 329, 343, 344, 359, 360, 375, 390, 391, 406, 407, 421, 422, 437, 438, 453, 468, 469, 484, 485, 500, 515, 516, 531, 547, 562, 578]],#大于100
 ['feat_26',[40, 28, 140, 52, 44, 48, 68, 65, 70, 73, 71, 63, 67, 69, 60, 59, 64, 72, 66, 61, 79, 62, 58, 75, 74, 76, 80, 78, 56, 57, 50, 81, 54, 77, 82, 93, 86, 106, 83, 88, 96, 112, 89, 105, 84, 53, 55, 92, 102, 100, 111, 97, 118, 104, 90, 132, 310, 98, 94, 99, 87, 129, 1478, 95, 103, 101, 91, 440, 85, 109, 353, 153, 32, 350, 119, 49, 114, 344, 311, 34, 444, 107, 422, 439, 116, 41, 51, 434, 437, 442, 167, 436, 435, 443, 427, 46, 433, 29, 438, 441]],#前100个特征
 ['feat_27',[0, 512, 1024, 8192, 29200, 4096, 65535, 64240, 26883, 16425, 28960, 10136, 11584, 14480, 15928, 16560, 229, 16698, 65160, 5792, 14600, 227, 24616, 26064, 20272, 21720, 65280, 7240, 8688, 50680, 1641, 16591, 74, 30408, 27512, 13032, 17376, 16384, 39096, 47784, 23168, 16660, 65170, 64800, 36200, 56472, 16383, 34752, 52128, 43440, 17520, 32850, 65340, 64860, 237, 16450, 41992, 33304, 16614, 44888, 60816, 64400, 65392, 5840, 1200, 16426, 64308, 243, 64492, 16500, 235, 16685, 251, 64952, 26847, 64896, 16695, 49232, 282, 16567, 16422, 64620, 8212, 63443, 16652, 296, 16575, 16473, 16468, 256, 1400, 259, 16440, 65534, 1026, 2053, 200, 274, 65320, 258]],#前100个特征         
 ['feat_28',[0, 4294951, 4294952, 4294936, 4293967, 4293842, 4293717, 4294092, 4293592, 4294014, 4293889, 4293920, 4293795, 4293764, 4293670, 4294139, 4294045, 4294217, 4293639, 4293748, 4293936, 4294920, 4293545, 4293998, 4293811, 4294123, 4294342, 4293686, 4293701, 4294295, 4293467, 4294264, 4293623, 4294061, 4293873, 4294467, 4294170, 4293826, 4293608, 4293514, 4294248, 4293951, 4294108, 4294076, 4294514, 4294186, 4293561, 4294545, 4294311, 4293733, 4293779, 4294389, 4293858, 4294592, 4293983, 4294201, 4294420, 4293576, 4294029, 4294498, 4293780, 4293498, 4294358, 4294436, 4294373, 4294030, 4294561, 4294154, 4294326, 4293342, 4294639, 4294233, 4293904, 4293905, 4293389, 4293655, 4294670, 4294155, 4293420, 4294279, 4294483, 4293483, 4293217, 4293702, 4293654, 4293529, 4293436, 4294935, 4293732, 4294686, 4294889, 4293530, 4294576, 4294904, 4294623, 4293982, 4294077, 4293373, 4294964, 4294451]],#取前100个value
 ['feat_29',[56, 280, 44, 84, 420, 48, 140, 112, 40, 68, 65, 164, 70, 71, 320, 180, 132, 63, 67, 60, 69, 64, 72, 73, 59, 66, 28, 1480, 1460, 232, 1476, 61, 79, 146, 62, 360, 75, 373, 58, 276, 498, 585, 74, 216, 328, 349, 52, 1560, 224, 130]],#取前50个value
 ['feat_30',[7.0, 0.0, 188.0, 91.0, 5.126, 1.0, 41.0, 91.178, 91.126, 7.178, 5.178, 5.119, 5.24, 77.0, 7.126, 92.0, 5.239, 5.2120000000000015, 3.0, 4.0, 88.0, 91.22, 5.12, 91.212, 131.0, 91.239, 91.119, 5.233, 91.24, 91.12, 5.0, 89.0, 2.0, 10.0, 91.14, 5.124, 1.178, 20.0, 29.0, 91.29, 91.233, 5.14, 114.0, 7.22, 5.231, 89.7, 158.0, 161.0, 100.0, 51.0]],#取前50个value
 
 ['feat_32',[0, 46, 181, 227, 229, 235, 237, 251, 512, 5792, 5840, 7240, 8192, 8688, 10136, 11584, 13032, 14480, 14600, 15928, 16383, 17376, 17922, 18824, 20272, 24616, 26847, 26883, 28200, 28400, 28960, 29200, 30016, 30408, 31088, 31856, 34752, 35300, 42780, 47784, 59368, 60720, 64000, 64240, 65160, 65340, 65535]],#大于100
 ['feat_33',[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 3600, 86400, 172800, 604800]],#大于100
 ['feat_34',[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 23, 25, 26, 27, 29, 31, 32, 35, 38, 196, 369, 389]],
 ['feat_35',[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 51, 58, 59, 60, 62, 64, 66, 67, 68, 70, 72, 74, 76, 78, 79, 80, 82, 86, 87, 90, 92, 94, 98, 99, 100, 104, 106, 110, 116, 120, 200, 222, 240]],      
 ['feat_36',[0, 320000, 352000, 14984000, 1056000, 8000, 648000, 13768000, 800000, 480000, 544000, 520000, 688000, 568000, 1424000, 536000, 9088000, 1336000, 5600000, 416000, 16000, 760000, 2736000, 2752000, 640000, 17144000, 704000, 672000, 6112000, 1040000, 24000, 712000, 2584000, 2288000, 1296000, 1024000, 736000, 1016000, 856000, 728000, 15304000, 14952000, 81344000, 2672000, 2432000, 1248000, 1280000, 960000, 1032000, 56000]],#取前50个value
 ['feat_38',[0, 10104, 38007, 53421]],#大于10
 ['feat_41',[64, 0, 128, 32, 127, 100, 101, 103, 102, 63, 31, 105, 99, 104, 98, 106, 97, 96, 107, 1, 108, 95, 109, 112, 234, 71, 111, 94, 110, 93, 255, 41, 36, 43, 92, 40, 59, 39, 91, 37, 254, 38, 35, 235, 30, 33, 34, 42, 47, 4, 237, 44, 113, 230, 232, 236, 90, 45, 49, 233, 231, 89, 240, 46, 58, 29, 57, 51, 228, 28, 88, 50, 55, 227, 229, 114, 48, 211, 54, 225, 224, 212, 87, 56, 218, 53, 52, 223, 210, 14, 217, 26, 24, 215, 3, 214, 239, 168, 220, 238]],#取前100个特征
 ['feat_42',[0,1,28,12,255,43,2]],#出现次数大于20次
]

def get_feats(df):
    for feat in feat_one_hot:
        for i in range(len(feat[1])):
            df[f"{feat[0]}_{feat[1][i]}"]=(df[feat[0]]==feat[1][i]).astype(np.int8)
    df['feat_3']=df['feat_3']/8000
    df['total_feat']=df['feat_0']-df['feat_2']+df['feat_4']-df['feat_8']-df['feat_26']+df['feat_27']-df['feat_28']-df['feat_41']

    tops=['feat_0','feat_2','feat_4','feat_8','feat_26','feat_27','feat_28','feat_30','feat_32','feat_41','feat_13']
    for i in range(len(tops)):
        for j in range(i+1,len(tops)):
            df[f"{tops[i]}+{tops[j]}"]=df[tops[i]]+df[tops[j]]
            df[f"{tops[i]}-{tops[j]}"]=df[tops[i]]-df[tops[j]]
            df[f"{tops[i]}*{tops[j]}"]=df[tops[i]]*df[tops[j]]
            df[f"{tops[i]}/{tops[j]}"]=df[tops[i]]/(df[tops[j]]+1e-15)
            
    tops=['feat_8','feat_26','feat_27','feat_28','feat_30','feat_32','feat_41','feat_10','feat_11','feat_12','feat_13']
    for i in range(len(tops)):
        for j in range(i+1,len(tops)):
            df[f"{tops[i]}+{tops[j]}"]=df[tops[i]]+df[tops[j]]
            df[f"{tops[i]}-{tops[j]}"]=df[tops[i]]-df[tops[j]]
            df[f"{tops[i]}*{tops[j]}"]=df[tops[i]]*df[tops[j]]
            df[f"{tops[i]}/{tops[j]}"]=df[tops[i]]/(df[tops[j]]+1e-15)
            
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    #512个迭代器出现少于5次
    df.drop(['feat_0', 'feat_0_6', 'feat_0_17', 'feat_0_1', 'feat_0_2', 'feat_0_58', 'feat_1_0', 'feat_1_4', 'feat_1_9', 'feat_1_10', 'feat_1_11', 'feat_1_12', 'feat_1_13', 'feat_1_14', 'feat_1_15', 'feat_1_16', 'feat_1_18', 'feat_1_20', 'feat_1_22', 'feat_1_24', 'feat_1_28', 'feat_1_30', 'feat_1_32', 'feat_1_36', 'feat_1_40', 'feat_1_42', 'feat_1_44', 'feat_1_48', 'feat_1_54', 'feat_1_60', 'feat_1_66', 'feat_1_352', 'feat_1_704', 'feat_1_720', 'feat_2_0', 'feat_2_31', 'feat_3_1056000', 'feat_3_560000', 'feat_3_536000', 'feat_3_480000', 'feat_3_552000', 'feat_3_512000', 'feat_3_11840000', 'feat_3_576000', 'feat_3_11680000', 'feat_3_472000', 'feat_3_584000', 'feat_3_528000', 'feat_3_11808000', 'feat_4_0', 'feat_4_40', 'feat_4_44', 'feat_4_52', 'feat_4_55', 'feat_4_56', 'feat_4_57', 'feat_4_58', 'feat_4_59', 'feat_4_60', 'feat_4_61', 'feat_4_62', 'feat_4_63', 'feat_4_64', 'feat_4_66', 'feat_4_69', 'feat_4_70', 'feat_4_71', 'feat_4_73', 'feat_4_74', 'feat_4_75', 'feat_4_76', 'feat_4_77', 'feat_4_78', 'feat_4_79', 'feat_4_80', 'feat_4_81', 'feat_4_82', 'feat_4_83', 'feat_4_84', 'feat_4_86', 'feat_4_88', 'feat_4_89', 'feat_4_90', 'feat_4_92', 'feat_4_93', 'feat_4_96', 'feat_4_97', 'feat_4_102', 'feat_4_104', 'feat_4_105', 'feat_4_106', 'feat_4_111', 'feat_4_112', 'feat_5_0', 'feat_5_41', 'feat_5_74', 'feat_5_82', 'feat_5_83', 'feat_5_118', 'feat_5_123', 'feat_5_125', 'feat_5_126', 'feat_5_128', 'feat_5_132', 'feat_5_136', 'feat_5_140', 'feat_5_143', 'feat_5_164', 'feat_5_166', 'feat_5_174', 'feat_5_177', 'feat_5_180', 'feat_5_187', 'feat_5_190', 'feat_5_195', 'feat_5_201', 'feat_5_205', 'feat_5_207', 'feat_5_208', 'feat_5_219', 'feat_5_239', 'feat_5_245', 'feat_5_246', 'feat_5_249', 'feat_5_251', 'feat_5_257', 'feat_5_261', 'feat_5_280', 'feat_5_285', 'feat_5_287', 'feat_5_289', 'feat_5_293', 'feat_5_299', 'feat_5_302', 'feat_5_303', 'feat_5_304', 'feat_5_305', 'feat_5_306', 'feat_5_307', 'feat_5_309', 'feat_5_310', 'feat_5_328', 'feat_5_329', 'feat_5_332', 'feat_5_348', 'feat_5_351', 'feat_5_354', 'feat_5_356', 'feat_5_357', 'feat_5_358', 'feat_5_359', 'feat_5_360', 'feat_5_361', 'feat_5_362', 'feat_5_363', 'feat_5_364', 'feat_5_365', 'feat_5_366', 'feat_5_367', 'feat_5_368', 'feat_5_369', 'feat_5_370', 'feat_5_372', 'feat_5_373', 'feat_5_374', 'feat_5_375', 'feat_5_376', 'feat_5_377', 'feat_5_378', 'feat_5_379', 'feat_5_380', 'feat_5_381', 'feat_5_382', 'feat_5_383', 'feat_5_384', 'feat_5_385', 'feat_5_386', 'feat_5_387', 'feat_5_390', 'feat_5_392', 'feat_5_393', 'feat_5_395', 'feat_5_396', 'feat_5_397', 'feat_5_398', 'feat_5_399', 'feat_5_400', 'feat_5_401', 'feat_5_402', 'feat_5_403', 'feat_5_404', 'feat_5_405', 'feat_5_406', 'feat_5_407', 'feat_5_408', 'feat_5_409', 'feat_5_410', 'feat_5_411', 'feat_5_412', 'feat_5_413', 'feat_5_416', 'feat_5_417', 'feat_5_418', 'feat_5_419', 'feat_5_420', 'feat_5_421', 'feat_5_422', 'feat_5_423', 'feat_5_424', 'feat_5_428', 'feat_5_429', 'feat_5_430', 'feat_5_431', 'feat_5_432', 'feat_5_433', 'feat_5_436', 'feat_5_451', 'feat_5_456', 'feat_5_462', 'feat_5_490', 'feat_5_492', 'feat_5_505', 'feat_5_506', 'feat_5_533', 'feat_5_539', 'feat_5_547', 'feat_5_555', 'feat_5_563', 'feat_5_574', 'feat_5_578', 'feat_5_605', 'feat_5_615', 'feat_5_621', 'feat_5_623', 'feat_5_626', 'feat_5_642', 'feat_5_644', 'feat_5_656', 'feat_5_658', 'feat_5_663', 'feat_5_674', 'feat_5_688', 'feat_5_690', 'feat_5_697', 'feat_5_701', 'feat_5_706', 'feat_5_711', 'feat_5_712', 'feat_5_713', 'feat_5_717', 'feat_5_722', 'feat_5_738', 'feat_5_779', 'feat_5_782', 'feat_5_784', 'feat_5_786', 'feat_5_795', 'feat_5_800', 'feat_5_820', 'feat_5_826', 'feat_5_842', 'feat_5_850', 'feat_5_875', 'feat_5_891', 'feat_5_920', 'feat_5_925', 'feat_5_945', 'feat_5_946', 'feat_5_952', 'feat_5_962', 'feat_5_989', 'feat_5_1140', 'feat_5_1340', 'feat_5_1402', 'feat_5_1434', 'feat_5_1435', 'feat_5_1472', 'feat_5_1541', 'feat_5_1660', 'feat_5_1665', 'feat_5_1676', 'feat_5_1679', 'feat_5_1680', 'feat_5_1684', 'feat_5_1692', 'feat_5_1712', 'feat_5_1744', 'feat_5_1752', 'feat_5_1756', 'feat_5_1764', 'feat_5_1768', 'feat_5_1812', 'feat_5_1820', 'feat_5_1836', 'feat_5_1860', 'feat_5_1868', 'feat_5_1912', 'feat_5_1968', 'feat_5_1980', 'feat_5_1982', 'feat_5_1992', 'feat_5_2024', 'feat_5_2028', 'feat_5_2064', 'feat_5_2068', 'feat_5_2076', 'feat_5_2088', 'feat_5_2132', 'feat_5_2140', 'feat_5_2234', 'feat_5_2235', 'feat_5_2236', 'feat_5_2237', 'feat_5_2238', 'feat_5_2239', 'feat_5_2240', 'feat_5_2241', 'feat_5_2242', 'feat_5_2243', 'feat_5_2244', 'feat_5_2632', 'feat_5_2820', 'feat_5_2852', 'feat_5_2961', 'feat_5_3144', 'feat_5_3240', 'feat_5_3537', 'feat_5_4025', 'feat_5_4252', 'feat_5_4812', 'feat_5_4948', 'feat_5_5244', 'feat_5_5744', 'feat_5_8440', 'feat_5_8944', 'feat_5_18117', 'feat_5_18118', 'feat_5_18119', 'feat_5_18123', 'feat_5_18133', 'feat_5_18138', 'feat_5_18144', 'feat_5_18149', 'feat_5_18159', 'feat_5_18160', 'feat_5_18162', 'feat_5_18194', 'feat_5_18208', 'feat_5_18223', 'feat_5_18282', 'feat_6_8', 'feat_6_25', 'feat_6_45', 'feat_6_14', 'feat_6_47', 'feat_6_76', 'feat_6_38', 'feat_6_42', 'feat_6_30', 'feat_6_67', 'feat_8_0', 'feat_8_35840', 'feat_8_49920', 'feat_8_6912', 'feat_8_6144', 'feat_8_8960', 'feat_8_11008', 'feat_8_16896', 'feat_8_62976', 'feat_8_13056', 'feat_8_52480', 'feat_8_33792', 'feat_8_64000', 'feat_8_63744', 'feat_8_48640', 'feat_8_44544', 'feat_8_10496', 'feat_8_39936', 'feat_8_31232', 'feat_8_46080', 'feat_8_32000', 'feat_8_18944', 'feat_8_21248', 'feat_8_13568', 'feat_8_64256', 'feat_8_1280', 'feat_8_32256', 'feat_8_20992', 'feat_8_41984', 'feat_8_62720', 'feat_10_80', 'feat_10_57044', 'feat_10_42640', 'feat_10_52071', 'feat_10_56304', 'feat_10_49731', 'feat_10_40081', 'feat_10_59460', 'feat_10_65233', 'feat_10_57184', 'feat_10_41607', 'feat_10_47422', 'feat_10_47095', 'feat_10_54114', 'feat_10_50428', 'feat_10_35063', 'feat_10_40850', 'feat_10_0', 'feat_10_54193', 'feat_10_53', 'feat_10_52970', 'feat_10_43411', 'feat_10_50099', 'feat_10_50519', 'feat_10_51598', 'feat_10_51103', 'feat_10_50227', 'feat_10_44486', 'feat_10_6000', 'feat_11_0', 'feat_11_27', 'feat_11_30', 'feat_11_31', 'feat_11_82', 'feat_11_86', 'feat_11_90', 'feat_11_91', 'feat_11_94', 'feat_11_95', 'feat_11_218', 'feat_12_28', 'feat_12_1500', 'feat_12_1213', 'feat_12_975', 'feat_12_81', 'feat_12_1189', 'feat_12_89', 'feat_12_1352', 'feat_12_1470', 'feat_12_987', 'feat_12_106', 'feat_12_86', 'feat_12_1219', 'feat_12_167', 'feat_12_1406', 'feat_12_171', 'feat_12_528', 'feat_13_0', 'feat_13_111', 'feat_13_44', 'feat_13_77', 'feat_13_31', 'feat_13_74', 'feat_13_211', 'feat_13_55', 'feat_13_15', 'feat_13_69', 'feat_13_39', 'feat_13_136', 'feat_13_61', 'feat_13_9', 'feat_13_101', 'feat_13_66', 'feat_14_0', 'feat_14_2', 'feat_14_23', 'feat_14_24', 'feat_14_41', 'feat_14_194', 'feat_14_211', 'feat_14_214', 'feat_14_219', 'feat_14_222', 'feat_14_223', 'feat_15_4', 'feat_15_5', 'feat_15_6', 'feat_15_7', 'feat_15_8', 'feat_15_18', 'feat_15_32', 'feat_15_64', 'feat_19_0.0', 'feat_19_125.0', 'feat_19_221.0', 'feat_19_229.0', 'feat_19_230.0', 'feat_19_331.0', 'feat_20_0', 'feat_20_3', 'feat_20_4', 'feat_20_5', 'feat_20_6', 'feat_20_7', 'feat_20_8', 'feat_20_9', 'feat_20_10', 'feat_20_11', 'feat_20_13', 'feat_20_16', 'feat_20_17', 'feat_20_18', 'feat_20_20', 'feat_20_29', 'feat_20_37', 'feat_21_0.0', 'feat_21_1873.0', 'feat_21_1147.0', 'feat_21_1721.0', 'feat_21_65.0', 'feat_21_86.0', 'feat_21_178.0', 'feat_21_71.0', 'feat_21_167.0', 'feat_21_60.0', 'feat_21_700.0', 'feat_21_2143.0', 'feat_21_344.0', 'feat_21_764.0', 'feat_21_342.0', 'feat_21_84.0', 'feat_22_1', 'feat_22_18', 'feat_22_19', 'feat_22_20', 'feat_22_22', 'feat_22_21', 'feat_22_27', 'feat_22_23', 'feat_22_26', 'feat_22_24', 'feat_22_52', 'feat_22_28', 'feat_22_25', 'feat_24_28', 'feat_24_1500', 'feat_24_1213', 'feat_24_40', 'feat_24_975', 'feat_24_52', 'feat_24_81', 'feat_24_1189', 'feat_24_89', 'feat_24_1352', 'feat_24_1470', 'feat_24_987', 'feat_24_106', 'feat_24_100', 'feat_24_86', 'feat_24_68', 'feat_24_1219', 'feat_24_71', 'feat_24_167', 'feat_24_67', 'feat_24_1406', 'feat_24_171', 'feat_24_528', 'feat_25_0', 'feat_25_1', 'feat_25_3', 'feat_25_4', 'feat_25_6', 'feat_25_7', 'feat_25_8', 'feat_25_9', 'feat_25_10', 'feat_25_11', 'feat_25_12', 'feat_25_19', 'feat_25_20', 'feat_25_21', 'feat_25_22', 'feat_25_23', 'feat_25_24', 'feat_25_25', 'feat_25_26', 'feat_25_27', 'feat_25_29', 'feat_25_30', 'feat_25_33', 'feat_25_38', 'feat_25_39', 'feat_25_41', 'feat_25_43', 'feat_25_79', 'feat_25_93', 'feat_25_109', 'feat_25_110', 'feat_25_140', 'feat_25_156', 'feat_25_157', 'feat_25_234', 'feat_25_265', 'feat_25_266', 'feat_25_281', 'feat_25_282', 'feat_25_297', 'feat_25_312', 'feat_25_391', 'feat_25_407', 'feat_25_438', 'feat_25_453', 'feat_25_484', 'feat_25_485', 'feat_25_500', 'feat_25_516', 'feat_25_547', 'feat_25_562', 'feat_26_28', 'feat_26_40', 'feat_26_48', 'feat_26_53', 'feat_26_55', 'feat_26_56', 'feat_26_57', 'feat_26_58', 'feat_26_59', 'feat_26_60', 'feat_26_61', 'feat_26_62', 'feat_26_63', 'feat_26_64', 'feat_26_65', 'feat_26_66', 'feat_26_67', 'feat_26_69', 'feat_26_70', 'feat_26_72', 'feat_26_73', 'feat_26_74', 'feat_26_75', 'feat_26_76', 'feat_26_77', 'feat_26_78', 'feat_26_79', 'feat_26_80', 'feat_26_81', 'feat_26_82', 'feat_26_83', 'feat_26_84', 'feat_26_86', 'feat_26_88', 'feat_26_89', 'feat_26_90', 'feat_26_92', 'feat_26_93', 'feat_26_96', 'feat_26_97', 'feat_26_100', 'feat_26_102', 'feat_26_104', 'feat_26_105', 'feat_26_106', 'feat_26_111', 'feat_26_112', 'feat_26_118', 'feat_26_132', 'feat_26_140', 'feat_27_0', 'feat_27_227', 'feat_27_243', 'feat_27_1200', 'feat_27_5792', 'feat_27_7240', 'feat_27_8688', 'feat_27_10136', 'feat_27_11584', 'feat_27_13032', 'feat_27_14480', 'feat_27_14600', 'feat_27_15928', 'feat_27_16384', 'feat_27_16425', 'feat_27_16426', 'feat_27_16450', 'feat_27_16560', 'feat_27_16591', 'feat_27_16614', 'feat_27_16660', 'feat_27_16698', 'feat_27_17376', 'feat_27_17520', 'feat_27_20272', 'feat_27_21720', 'feat_27_23168', 'feat_27_24616', 'feat_27_26064', 'feat_27_27512', 'feat_27_28960', 'feat_27_30408', 'feat_27_32850', 'feat_27_33304', 'feat_27_34752', 'feat_27_36200', 'feat_27_39096', 'feat_27_41992', 'feat_27_43440', 'feat_27_44888', 'feat_27_47784', 'feat_27_50680', 'feat_27_52128', 'feat_27_56472', 'feat_27_60816', 'feat_27_64240', 'feat_27_64308', 'feat_27_64400', 'feat_27_64800', 'feat_27_64860', 'feat_27_65160', 'feat_27_65170', 'feat_27_65280', 'feat_27_65340', 'feat_27_65392', 'feat_27_65535', 'feat_28_0', 'feat_28_4293889', 'feat_28_4294139', 'feat_28_4293998', 'feat_29_40', 'feat_29_65', 'feat_29_70', 'feat_29_132', 'feat_29_63', 'feat_29_67', 'feat_29_60', 'feat_29_69', 'feat_29_64', 'feat_29_72', 'feat_29_73', 'feat_29_59', 'feat_29_66', 'feat_29_28', 'feat_29_1480', 'feat_29_1460', 'feat_29_232', 'feat_30_0.0', 'feat_30_5.126', 'feat_30_91.178', 'feat_30_91.126', 'feat_30_5.178', 'feat_30_5.119', 'feat_30_5.24', 'feat_30_77.0', 'feat_30_7.126', 'feat_30_5.239', 'feat_30_5.2120000000000015', 'feat_30_91.22', 'feat_30_5.12', 'feat_30_91.212', 'feat_30_91.239', 'feat_30_91.119', 'feat_30_5.233', 'feat_30_91.24', 'feat_30_91.12', 'feat_32_0', 'feat_32_46', 'feat_32_181', 'feat_32_227', 'feat_32_229', 'feat_32_7240', 'feat_32_8192', 'feat_32_8688', 'feat_32_10136', 'feat_32_11584', 'feat_32_13032', 'feat_32_14480', 'feat_32_14600', 'feat_32_15928', 'feat_32_17376', 'feat_32_17922', 'feat_32_18824', 'feat_32_20272', 'feat_32_24616', 'feat_32_28200', 'feat_32_28400', 'feat_32_29200', 'feat_32_30016', 'feat_32_30408', 'feat_32_31088', 'feat_32_31856', 'feat_32_34752', 'feat_32_35300', 'feat_32_42780', 'feat_32_47784', 'feat_32_59368', 'feat_32_60720', 'feat_32_64000', 'feat_32_65160', 'feat_32_65340', 'feat_32_65535', 'feat_33_0', 'feat_33_2', 'feat_33_5', 'feat_33_6', 'feat_33_9', 'feat_33_11', 'feat_33_16', 'feat_33_18', 'feat_33_19', 'feat_33_23', 'feat_33_26', 'feat_33_29', 'feat_33_31', 'feat_33_32', 'feat_33_34', 'feat_33_35', 'feat_33_36', 'feat_33_39', 'feat_33_40', 'feat_33_44', 'feat_33_45', 'feat_33_46', 'feat_33_50', 'feat_33_59', 'feat_33_3600', 'feat_33_86400', 'feat_33_172800', 'feat_33_604800', 'feat_34_2', 'feat_34_3', 'feat_34_4', 'feat_34_5', 'feat_34_6', 'feat_34_7', 'feat_34_8', 'feat_34_9', 'feat_34_10', 'feat_34_11', 'feat_34_12', 'feat_34_13', 'feat_34_14', 'feat_34_15', 'feat_34_16', 'feat_34_17', 'feat_34_19', 'feat_34_20', 'feat_34_21', 'feat_34_23', 'feat_34_25', 'feat_34_26', 'feat_34_27', 'feat_34_29', 'feat_34_31', 'feat_34_32', 'feat_34_35', 'feat_34_38', 'feat_34_196', 'feat_34_369', 'feat_34_389', 'feat_35_0', 'feat_35_1', 'feat_35_21', 'feat_35_23', 'feat_35_24', 'feat_35_26', 'feat_35_27', 'feat_35_28', 'feat_35_29', 'feat_35_30', 'feat_35_31', 'feat_35_32', 'feat_35_33', 'feat_35_34', 'feat_35_36', 'feat_35_37', 'feat_35_38', 'feat_35_39', 'feat_35_40', 'feat_35_41', 'feat_35_42', 'feat_35_43', 'feat_35_44', 'feat_35_46', 'feat_35_51', 'feat_35_58', 'feat_35_59', 'feat_35_60', 'feat_35_62', 'feat_35_64', 'feat_35_66', 'feat_35_67', 'feat_35_68', 'feat_35_70', 'feat_35_72', 'feat_35_74', 'feat_35_76', 'feat_35_78', 'feat_35_79', 'feat_35_80', 'feat_35_82', 'feat_35_86', 'feat_35_87', 'feat_35_90', 'feat_35_92', 'feat_35_94', 'feat_35_98', 'feat_35_99', 'feat_35_100', 'feat_35_104', 'feat_35_106', 'feat_35_110', 'feat_35_116', 'feat_35_120', 'feat_35_222', 'feat_35_240', 'feat_36_0', 'feat_36_14984000', 'feat_36_648000', 'feat_36_13768000', 'feat_36_800000', 'feat_36_480000', 'feat_36_688000', 'feat_36_1424000', 'feat_36_9088000', 'feat_36_1336000', 'feat_36_5600000', 'feat_36_760000', 'feat_36_2736000', 'feat_36_2752000', 'feat_36_17144000', 'feat_36_672000', 'feat_36_6112000', 'feat_36_1040000', 'feat_38_10104', 'feat_38_38007', 'feat_38_53421', 'feat_41_0', 'feat_41_31', 'feat_41_36', 'feat_41_37', 'feat_41_38', 'feat_41_39', 'feat_41_40', 'feat_41_41', 'feat_41_43', 'feat_41_59', 'feat_41_91', 'feat_41_92', 'feat_41_93', 'feat_41_94', 'feat_41_95', 'feat_41_96', 'feat_41_97', 'feat_41_98', 'feat_41_99', 'feat_41_100', 'feat_41_101', 'feat_41_102', 'feat_41_103', 'feat_41_104', 'feat_41_105', 'feat_41_106', 'feat_41_107', 'feat_41_108', 'feat_41_109', 'feat_41_110', 'feat_41_111', 'feat_41_112', 'feat_41_234', 'feat_41_254', 'feat_41_255', 'feat_42_0', 'feat_42_43', 'feat_42_255', 'feat_0+feat_8', 'feat_0*feat_8', 'feat_0*feat_27', 'feat_0*feat_32', 'feat_4*feat_26', 'feat_4/feat_26', 'feat_8*feat_41', 'feat_26*feat_32', 'feat_8*feat_11'],axis=1,inplace=True)

    return df

train=get_feats(train)
test=get_feats(test)

for i in range(43):
    if f'feat_{i}' in list(train.columns):
        feat_dict=train[f'feat_{i}'].value_counts().to_dict()
        train[f'feat_{i}_value_counts']=train[f'feat_{i}'].apply(lambda x:feat_dict.get(x,0))
        test[f'feat_{i}_value_counts']=test[f'feat_{i}'].apply(lambda x:feat_dict.get(x,0))
        
tops=['feat_2_value_counts','feat_3_value_counts','feat_5_value_counts','feat_6_value_counts','feat_8_value_counts',
'feat_12_value_counts','feat_20_value_counts','feat_22_value_counts','feat_24_value_counts','feat_28_value_counts','feat_29_value_counts',
]
for i in range(len(tops)):
    for j in range(i+1,len(tops)):
        train[f"{tops[i]}+{tops[j]}"]=train[tops[i]]+train[tops[j]]
        train[f"{tops[i]}-{tops[j]}"]=train[tops[i]]-train[tops[j]]
        train[f"{tops[i]}*{tops[j]}"]=train[tops[i]]*train[tops[j]]
        train[f"{tops[i]}/{tops[j]}"]=train[tops[i]]/(train[tops[j]]+1e-15)
        
        test[f"{tops[i]}+{tops[j]}"]=test[tops[i]]+test[tops[j]]
        test[f"{tops[i]}-{tops[j]}"]=test[tops[i]]-test[tops[j]]
        test[f"{tops[i]}*{tops[j]}"]=test[tops[i]]*test[tops[j]]
        test[f"{tops[i]}/{tops[j]}"]=test[tops[i]]/(test[tops[j]]+1e-15)
        
drop_cols=['feat_8', 'feat_20', 'feat_26', 'feat_27', 'feat_32', 'feat_1_8', 'feat_2_16', 'feat_2_15', 'feat_2_1000', 'feat_2_875', 'feat_2_953', 'feat_2_1078', 'feat_2_1047', 'feat_2_1172', 'feat_2_1297', 'feat_2_828', 'feat_2_922', 'feat_2_750', 'feat_2_1422', 'feat_2_844', 'feat_2_625', 'feat_2_672', 'feat_2_703', 'feat_3_1120000', 'feat_3_224000', 'feat_3_352000', 'feat_3_544000', 'feat_3_520000', 'feat_3_1168000', 'feat_3_40000', 'feat_4_53', 'feat_4_65', 'feat_4_67', 'feat_4_68', 'feat_5_100', 'feat_6_80', 'feat_10_443', 'feat_11_16', 'feat_12_71', 'feat_12_67', 'feat_12_180', 'feat_13_20', 'feat_14_6', 'feat_14_16', 'feat_14_28', 'feat_20_1', 'feat_21_132.0', 'feat_21_100.0', 'feat_21_95.0', 'feat_22_2', 'feat_22_11', 'feat_24_180', 'feat_25_2', 'feat_25_5', 'feat_25_15', 'feat_25_16', 'feat_25_17', 'feat_25_31', 'feat_25_46', 'feat_25_47', 'feat_25_63', 'feat_25_78', 'feat_25_125', 'feat_25_141', 'feat_25_187', 'feat_25_218', 'feat_25_219', 'feat_25_235', 'feat_25_250', 'feat_25_313', 'feat_25_329', 'feat_25_343', 'feat_25_360', 'feat_25_375', 'feat_25_390', 'feat_25_421', 'feat_25_437', 'feat_25_468', 'feat_25_469', 'feat_25_515', 'feat_25_531', 'feat_25_578', 'feat_26_52', 'feat_26_68', 'feat_28_4294951', 'feat_28_4294936', 'feat_28_4293967', 'feat_28_4293717', 'feat_28_4294092', 'feat_28_4294014', 'feat_28_4293920', 'feat_28_4293795', 'feat_28_4293670', 'feat_28_4294045', 'feat_28_4294217', 'feat_28_4294920', 'feat_28_4293545', 'feat_28_4294123', 'feat_28_4294342', 'feat_28_4293686', 'feat_28_4293701', 'feat_28_4294295', 'feat_29_56', 'feat_29_280', 'feat_29_44', 'feat_29_420', 'feat_29_112', 'feat_29_68', 'feat_29_71', 'feat_30_131.0', 'feat_32_251', 'feat_33_3', 'feat_33_7', 'feat_33_8', 'feat_33_12', 'feat_33_13', 'feat_33_14', 'feat_33_17', 'feat_33_20', 'feat_33_22', 'feat_33_25', 'feat_33_33', 'feat_33_42', 'feat_33_43', 'feat_33_47', 'feat_33_48', 'feat_33_51', 'feat_33_53', 'feat_33_55', 'feat_33_56', 'feat_33_58', 'feat_34_0', 'feat_35_12', 'feat_35_14', 'feat_35_17', 'feat_35_18', 'feat_35_19', 'feat_35_25', 'feat_35_35', 'feat_36_1056000', 'feat_36_568000', 'feat_36_16000', 'feat_38_0', 'feat_41_1', 'feat_0*feat_4', 'feat_0-feat_8', 'feat_0-feat_27', 'feat_8*feat_27', 'feat_8/feat_41', 'feat_8/feat_11', 'feat_5_value_counts', 'feat_20_value_counts', 'feat_5_value_counts+feat_20_value_counts', 'feat_5_value_counts*feat_20_value_counts', 'feat_5_value_counts-feat_24_value_counts', 'feat_8_value_counts+feat_24_value_counts', 'feat_8_value_counts-feat_24_value_counts', 'feat_8_value_counts/feat_24_value_counts', 'feat_12_value_counts+feat_20_value_counts', 'feat_12_value_counts+feat_24_value_counts', 'feat_12_value_counts-feat_24_value_counts', 'feat_12_value_counts*feat_24_value_counts', 'feat_12_value_counts/feat_24_value_counts', 'feat_20_value_counts+feat_24_value_counts', 'feat_20_value_counts-feat_24_value_counts']
train.drop(drop_cols,axis=1,inplace=True,errors='ignore')
test.drop(drop_cols,axis=1,inplace=True,errors='ignore')
train.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_10,feat_11,feat_12,feat_13,...,feat_24_value_counts*feat_28_value_counts,feat_24_value_counts/feat_28_value_counts,feat_24_value_counts+feat_29_value_counts,feat_24_value_counts-feat_29_value_counts,feat_24_value_counts*feat_29_value_counts,feat_24_value_counts/feat_29_value_counts,feat_28_value_counts+feat_29_value_counts,feat_28_value_counts-feat_29_value_counts,feat_28_value_counts*feat_29_value_counts,feat_28_value_counts/feat_29_value_counts
0,0,656,0.0,0,140,0,5612,0,140,0,...,409360400,86.137615,320916,54644,25000278080,1.410437,135316,-130956,290236480,0.016374
1,0,0,140.0,70,0,1,57621,0,170,1,...,153060267,0.001136,6841,-6007,2678808,0.064913,373475,360627,2357935624,57.137453
2,0,921,28.0,0,0,0,54105,0,28,2,...,124061014,304.785266,340543,48363,28407638770,1.331049,146728,-145452,93205420,0.004367
3,0,0,44.0,40,0,2,49546,20,44,3,...,27401825354,0.203389,148585,723,5519244874,1.009779,440982,293120,27136447481,4.964778
4,0,0,57.0,57,0,3,54162,0,145,4,...,138378227,0.001027,1072,-318,262015,0.542446,367746,366356,255100445,528.130935


In [4]:
#遍历表格df的所有列修改数据类型减少内存使用
def reduce_mem_usage(df, float16_as32=True):
    #memory_usage()是df每列的内存使用量,sum是对它们求和, B->KB->MB
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:#遍历每列的列名
        col_type = df[col].dtype#列名的type
        if col_type != object and str(col_type)!='category':#不是object也就是说这里处理的是数值类型的变量
            c_min,c_max = df[col].min(),df[col].max() #求出这列的最大值和最小值
            if str(col_type)[:3] == 'int':#如果是int类型的变量,不管是int8,int16,int32还是int64
                #如果这列的取值范围是在int8的取值范围内,那就对类型进行转换 (-128 到 127)
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                #如果这列的取值范围是在int16的取值范围内,那就对类型进行转换(-32,768 到 32,767)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                #如果这列的取值范围是在int32的取值范围内,那就对类型进行转换(-2,147,483,648到2,147,483,647)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                #如果这列的取值范围是在int64的取值范围内,那就对类型进行转换(-9,223,372,036,854,775,808到9,223,372,036,854,775,807)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:#如果是浮点数类型.
                #如果数值在float16的取值范围内,如果觉得需要更高精度可以考虑float32
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:#如果数据需要更高的精度可以选择float32
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)  
                #如果数值在float32的取值范围内，对它进行类型转换
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                #如果数值在float64的取值范围内，对它进行类型转换
                else:
                    df[col] = df[col].astype(np.float64)
    #计算一下结束后的内存
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #相比一开始的内存减少了百分之多少
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
train=reduce_mem_usage(train, float16_as32=False)
test=reduce_mem_usage(test, float16_as32=False)

Memory usage of dataframe is 4153.44 MB
Memory usage after optimization is: 2454.38 MB
Decreased by 40.9%
Memory usage of dataframe is 1036.83 MB
Memory usage after optimization is: 613.02 MB
Decreased by 40.9%


In [5]:
for col in test.columns:
    skew=train[col].skew()
    if abs(skew)>0.5:
        min_value=train[col].min()
        train[col]=train[col]-min_value
        test[col]=test[col]-min_value
        
        #print(f"col:{col},skew:{skew}")
        train[col]=np.log1p(train[col])
        test[col]=np.log1p(test[col])

In [6]:
#model lgb分类模型,日志评估,早停防止过拟合
from  lightgbm import LGBMClassifier,log_evaluation
def accuracy(y_true,y_pred):
    return np.mean(y_true==y_pred)
#Best is trial 5 with value: 0.99523.

train_X=train.drop(['label'],axis=1).values
train_y=train['label'].values
param = {
 "boosting_type": "gbdt",
 'metric': 'multi_logloss', 
 'random_state': 2024, 'n_estimators': 1280, 
 'reg_alpha': 0.8181763815293407, 'reg_lambda': 0.22461456065021287,
 'colsample_bytree': 0.8000766410545448, 'subsample': 0.8774901510794015, 
 'learning_rate': 0.01314618929151126, 'num_leaves': 58, 'min_child_samples': 92,
 'device':'gpu','gpu_use_dp':True,#这行GPU环境的参数,想在CPU环境下运行注释这行代码
}
model = LGBMClassifier(**param) 

model.fit(train_X,train_y,eval_set=[(train_X,train_y)],callbacks=[log_evaluation(10)],)

test_X=test.values
test_preds=model.predict(test_X)
print(test_preds[:10])

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 125533
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 1222
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 501 dense feature groups (384.52 MB) transferred to GPU in 0.221667 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score -1.442595
[LightGBM] [Info] Start training from score -1.099523
[LightGBM] [Info] Start training from score -1.243203
[LightGBM] [Info] Start training from score -3.356846
[LightGBM] [Info] Start training from score -4.689775
[LightGBM] [Info] Start training from score -2.987615
[LightGBM] [Info] Start training from score -3.425400
[LightGBM] [Info] Start training from score -4.185145
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

In [7]:
submission=pd.read_csv(path+"sample_submit.csv")
submission['label']=test_preds
submission.to_csv("top2solution.csv",index=None)
submission.head()

,id,label
0,800000,0
1,800001,2
2,800002,1
3,800003,1
4,800004,1


## 后续上分思路:

#### 1.optuna进一步调参,找到更好的lgb模型参数。

#### 2.构造更多的one-hot特征并进行特征筛选。

#### 3.比赛可以使用2个模型,可以考虑使用使用xgb或者catboost等模型。